In [1]:
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

In [ ]:
def extract_intactness(dataframe, GEE_Intactness_Layer = intactness_layer,GEE_Intactness_ID = "constant",Biome="None"):
    intactness = ee.Image(GEE_Intactness_Layer)
    df= dataframe
    # Function to process a chunk of the DataFrame
    def process_chunk(chunk):
        # Create a FeatureCollection from the chunk
        features = ee.FeatureCollection([
            ee.Feature(ee.Geometry.Point([lon, lat])) for lon, lat in zip(chunk['longitude'], chunk['latitude'])
        ])

        # Sample the intactness values at the specified locations
        intactness_samples = intactness.sampleRegions(
            collection=features,
            scale=250,  # Scale in meters (resolution of the SRTM dataset)
            geometries=True  # Include geometry in the output
        )

        # Convert the sampled data to a Python list of dictionaries
        intactness_samples_list = intactness_samples.getInfo()['features']

        # Process the results into a list of dictionaries
        chunk_data = []
        for feature in intactness_samples_list:
            coordinates = feature['geometry']['coordinates']
        
            intactness_value = feature['properties'][GEE_Intactness_ID]
            chunk_data.append({
                'Longitude': coordinates[0],
                'Latitude': coordinates[1],
                'intactness': intactness_value
            })

        return chunk_data

    # Split the DataFrame into chunks of 100 rows
    chunk_size = 100
    chunks = [df[i:i + chunk_size] for i in range(0, df.shape[0], chunk_size)]

    # Process each chunk and combine the results
    intactness_data = []
    for chunk in chunks:
        print(f"Processing chunk with {len(chunk)} rows...")
        chunk_data = process_chunk(chunk)
        intactness_data.extend(chunk_data)

    # Create a DataFrame from the combined results
    intactness_df = pd.DataFrame(intactness_data)

    # Print the DataFrame
    # print(intactness_df)
    intactness_df["Biome"] = Biome
    return intactness_df


In [ ]:

def generate_random_points(region, num_points):
    # Generate random points within the region
    random_points = ee.FeatureCollection.randomPoints(region=region, points=num_points, seed=42)

    # Convert FeatureCollection to a list of dictionaries
    points_list = random_points.map(lambda f: ee.Feature(f).set('coords', f.geometry().coordinates())).getInfo()
    # Extract coordinates and create a DataFrame
    coordinates = [{'longitude': f['geometry']['coordinates'][0], 'latitude': f['geometry']['coordinates'][1]} for f in points_list['features']]
    df = pd.DataFrame(coordinates)
    return df 

In [ ]:
def boxplot_plot(dataframe, title):
  plt.figure(figsize=(10, 6))
  sns.boxplot(data=dataframe, x='status', y='intactness', palette='Set2', showfliers=False)
  plt.title(title, fontsize=14)
  plt.xlabel('Status', fontsize=12)
  plt.ylabel('Intactness', fontsize=12)
  plt.grid(axis='y', linestyle='--', alpha=0.7)
  sns.despine()
  plt.show()